# Basic Configs

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [3]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 128
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [ ]:
# !rm -rf project

In [4]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 253 (delta 132), reused 132 (delta 54), pack-reused 0
Receiving objects: 100% (253/253), 1.60 MiB | 22.18 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [5]:
!mv IDL-Project project

# Load Functions

In [6]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [7]:
trainset, trainloader, testset, testloader = get_data(batch_size)

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load Models

In [8]:
from project.summarize.classifier import ResidualBlock, ResNet18, cifar_resnet20, ResNet50, VGG

In [9]:
resnet18 = ResNet18(ResidualBlock)
resnet20 = cifar_resnet20('cifar10')  # With pretrained model
resnet50 = ResNet50()
vgg16 = VGG('VGG16')

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar10-resnet20-30abc31d.pth" to /root/.cache/torch/hub/checkpoints/cifar10-resnet20-30abc31d.pth


### Load Trained Models

In [10]:
vgg16_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/vgg16.pth"
vgg16.load_state_dict(torch.load(vgg16_path)['model_state_dict'])

resnet18_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/ResNet18.pth"
resnet18.load_state_dict(torch.load(resnet18_path)['model_state_dict'])

resnet50_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/Copy of ResNet50_49.pth"
resnet50.load_state_dict(torch.load(resnet50_path)['model_state_dict'])

<All keys matched successfully>

In [11]:
models = {
    'vgg16': vgg16,
    'resnet18': resnet18,
    'resnet20': resnet20,
    'resnet50': resnet50,
}

# Generate the Adversarial Samples

In [12]:
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 890kB 4.2MB/s 
     |████████████████████████████████| 235kB 19.2MB/s 
     |████████████████████████████████| 21.0MB 1.2MB/s 
     |████████████████████████████████| 747kB 46.9MB/s 


In [33]:
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescentPyTorch, CarliniLInfMethod, DeepFool
from art.estimators.classification import PyTorchClassifier

In [37]:
from project.summarize.normal_train import evaluate

## Prepare Attacks

## Eps = 0.03

In [31]:
def generate_attack(test_loader, classifier, attack):
    adv_examples = []
    # Loop over all examples in test set
    for image, target in test_loader:
        # Send the data and label to the device
        # image, target = image.to(device), target.to(device)

        # Call Attack
        adv_data = attack.generate(x=image, y=target)

        # adv_data = adv_data.squeeze().detach().cpu().numpy()
        # adv_examples.append( (target.flatten().detach().cpu().numpy(), adv_data) )
        adv_examples.append( (target.flatten().numpy(), adv_data) )
        # pred_list.append((init_pred.flatten().detach().cpu().numpy(), final_pred.flatten().detach().cpu().numpy()))
    
    label = [j for i in adv_examples for j in i[0]]
    adv_ex = [j for i in adv_examples for j in i[1]]

    dataset = AdvDataset(adv_ex, label)
    loader = data.DataLoader(dataset,
                             batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)

    # Return the accuracy and an adversarial example
    return dataset, loader

In [41]:
samples = {}
criterion = nn.CrossEntropyLoss()
shape = trainset[0][0].shape
eps = 0.03

for model_name, model in models.items():
    model.to(device)
    model.eval()
    classifier = PyTorchClassifier(
        model=model,
        loss=criterion,
        input_shape=shape,
        nb_classes=10)
    attack_samples = {}

    print('model name = ', model_name)
    print('FGM')
    attack = FastGradientMethod(estimator=classifier, eps=eps)
    dataset, loader = generate_attack(testloader, classifier, attack)
    attack_samples['FGM'] = dataset, loader

    _, acc = evaluate(model, loader, criterion, device)
    print('acc = ', acc)

    print('PGD')
    attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=eps, eps_step=0.01, max_iter=10)
    dataset, loader = generate_attack(testloader, classifier, attack)
    attack_samples['PGD'] = dataset, loader

    _, acc = evaluate(model, loader, criterion, device)
    print('acc = ', acc)

    print('C&W')
    attack = CarliniLInfMethod(classifier=classifier, eps=eps, max_iter=10)
    dataset, loader = generate_attack(testloader, classifier, attack)
    attack_samples['C&W'] = dataset, loader

    _, acc = evaluate(model, loader, criterion, device)
    print('acc = ', acc)

    # print('DeepFool')
    # attack = DeepFool(classifier=classifier, max_iter=10, epsilon=eps)
    # dataset, loader = generate_attack(testloader, classifier, attack)
    # attack_samples['DeepFool'] = dataset, loader

    # _, acc = evaluate(model, loader, criterion, device)
    # print('acc = ', acc)

    model.to(cpu)
    del classifier
    torch.cuda.empty_cache()
    
    samples[model_name] = attack_samples

model name =  vgg16
FGM
acc =  37.830000000000005
PGD


PGD - Batches:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/art/estimators/classification/pytorch.py:525: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_grad = torch.tensor(x).to(self._device)
/usr/local/lib/python3.6/dist-packages/art/estimators/classification/pytorch.py:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_grad = torch.tensor(y).to(self._device)


acc =  16.66
C&W


C&W L_inf: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


acc =  19.68
model name =  resnet18
FGM
acc =  43.519999999999996
PGD


...

C&W L_inf: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


acc =  3.54
model name =  resnet50
FGM
acc =  40.589999999999996
PGD


acc =  20.3
C&W


C&W L_inf: 100%|██████████| 1/1 [00:03<00:00,  3.82s/it]


acc =  22.09


# 4x4 Matrix

In [42]:
matrix = {}

for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, attack_samples in samples.items():
        print('classifier: ', model_name, ';  adversarial data comes from: ', adv_name)
        line[adv_name] = {}
        for attack_name, (dataset, loader) in attack_samples.items():
            loss, acc = evaluate(model, loader, criterion, device)
            line[adv_name][attack_name] = acc
            print('attack: ', attack_name, ';  acc = ', acc)
        print('-' * 20)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  vgg16 ;  adversarial data comes from:  vgg16
attack:  FGM ;  acc =  37.830000000000005
attack:  PGD ;  acc =  16.66
attack:  C&W ;  acc =  19.68
--------------------
classifier:  vgg16 ;  adversarial data comes from:  resnet18
attack:  FGM ;  acc =  82.39
attack:  PGD ;  acc =  82.03
attack:  C&W ;  acc =  89.75
--------------------
classifier:  vgg16 ;  adversarial data comes from:  resnet20
attack:  FGM ;  acc =  85.39
attack:  PGD ;  acc =  84.92
attack:  C&W ;  acc =  91.42
--------------------
classifier:  vgg16 ;  adversarial data comes from:  resnet50
attack:  FGM ;  acc =  82.64
attack:  PGD ;  acc =  81.89999999999999
attack:  C&W ;  acc =  90.02
--------------------
classifier:  resnet18 ;  adversarial data comes from:  vgg16
attack:  FGM ;  acc =  83.05
attack:  PGD ;  acc =  83.21
attack:  C&W ;  acc =  88.52
--------------------
classifier:  resnet18 ;  adversarial data comes from:  resnet18
attack:  FGM ;  acc =  43.519999999999996
attack:  PGD ;  acc =  32.3